In [54]:
import pandas as pd
import nltk 
from nltk.stem import WordNetLemmatizer
import gensim
from gensim.models import Doc2Vec
import pickle

In [55]:
df_NM= pd.read_csv("D:\FINAL_PM_Modi_Speech_Cleaned_string.csv")

In [56]:
df_NM.head()

,index,Date,Speech,Title,Cleaned_Speech,Cleaned_Speech_string
0,NMS1,2017-01-03,"Governor of Andhra Pradesh, Shri E. S. L. Nara...",PM's Address at the Inauguration of the 104th ...,['governor andhra pradesh shri e l narasimhan ...,governor andhra pradesh shri e l narasimhan ch...
1,NMS2,2017-01-09,Chief Minister of Gujarat Shri Vijay Rupani ji...,Text of PM's address at the Inauguration of No...,['chief minister gujarat shri vijay rupani ji ...,chief minister gujarat shri vijay rupani ji co...
2,NMS3,2017-01-09,I am delighted to be here at Gift City to ina...,Text of PM’s address on the occasion of Inaugu...,['delighted gift city inaugurate india first i...,delighted gift city inaugurate india first int...
3,NMS4,2017-02-01,I congratulate the Finance Minister Arun Jaitl...,English rendering of the Prime Minister’s Stat...,['congratulate finance minister arun jaitley j...,congratulate finance minister arun jaitley jee...
4,NMS5,2017-02-05,Sri Pejavar Math’s most respected Sri Vishwesh...,English rendering of the text of PM’s address ...,['sri pejavar math respected sri vishwesh tirt...,sri pejavar math respected sri vishwesh tirth ...


In [57]:
df_MS= pd.read_csv("D:\PM_MANMOHAN_Speech_Cleaned_string.csv")

In [58]:
df_MS = df_MS.drop('Location', axis=1)

In [59]:
df_MS.head()

,index,Date,Title,Speech,Cleaned_Speech,Cleaned_Speech_string
0,MSS1,2004-06-24,Prime Minister 's Address to the Nation,"My Fellow Citizens,I speak to you with a deep...",['fellow citizen speak deep sense humility ful...,fellow citizen speak deep sense humility fully...
1,MSS2,2004-06-29,PM's inaugural address at the Chief Ministers'...,"""I welcome you all to this Conference which w...",['welcome conference take stock effort rural d...,welcome conference take stock effort rural dev...
2,MSS3,2004-07-03,"PM's inaugural address at the ""Dialogue on Min...","""It gives me great pleasure to be amongst you...","['give great pleasure amongst morning', 'parti...",give great pleasure amongst morning particular...
3,MSS4,2004-07-26,PM's speech at the CSIR Society Meeting,"Distinguished Members of the CSIR Society, La...",['distinguished member csir society lady gentl...,distinguished member csir society lady gentlem...
4,MSS5,2004-07-31,PM’ statement at the Press Conference,"\n\t""My two days in Bangkok have been very use...","['two day bangkok useful productive', 'first b...",two day bangkok useful productive first bimste...


In [7]:
df_MS.shape

(1401, 6)

In [8]:
df_NM.shape

(1182, 6)

In [60]:
meta_df = pd.concat([df_MS, df_NM])
meta_df.reset_index(drop=True, inplace=True)
meta_df

,index,Date,Title,Speech,Cleaned_Speech,Cleaned_Speech_string
0,MSS1,2004-06-24,Prime Minister 's Address to the Nation,"My Fellow Citizens,I speak to you with a deep...",['fellow citizen speak deep sense humility ful...,fellow citizen speak deep sense humility fully...
1,MSS2,2004-06-29,PM's inaugural address at the Chief Ministers'...,"""I welcome you all to this Conference which w...",['welcome conference take stock effort rural d...,welcome conference take stock effort rural dev...
2,MSS3,2004-07-03,"PM's inaugural address at the ""Dialogue on Min...","""It gives me great pleasure to be amongst you...","['give great pleasure amongst morning', 'parti...",give great pleasure amongst morning particular...
3,MSS4,2004-07-26,PM's speech at the CSIR Society Meeting,"Distinguished Members of the CSIR Society, La...",['distinguished member csir society lady gentl...,distinguished member csir society lady gentlem...
4,MSS5,2004-07-31,PM’ statement at the Press Conference,"\n\t""My two days in Bangkok have been very use...","['two day bangkok useful productive', 'first b...",two day bangkok useful productive first bimste...
...,...,...,...,...,...,...
2578,NMS1178,2023-04-09,English rendering of PM’s address at the inaug...,My colleagues in the Union Cabinet Shri Bhupen...,['colleague union cabinet shri bhupender yadav...,colleague union cabinet shri bhupender yadav j...
2579,NMS1179,2023-04-12,English rendering of PM’s address at training ...,Namaskar! \n\nThe campaign to provide governme...,"['namaskar', 'campaign provide government job ...",namaskar campaign provide government job youth...
2580,NMS1180,2023-04-12,English rendering of PM’s address during flagg...,"Namaskar, Governor of Rajasthan Shri Kalraj Mi...",['namaskar governor rajasthan shri kalraj mish...,namaskar governor rajasthan shri kalraj mishra...
2581,NMS1181,2023-04-14,English rendering of PM’s address at inaugurat...,"Governor of Assam Shri Gulab Chand Kataria ji,...",['governor assam shri gulab chand kataria ji c...,governor assam shri gulab chand kataria ji chi...


In [61]:
meta_df.shape

(2583, 6)

In [70]:
meta_df['Cleaned_Speech_string'] = meta_df['Cleaned_Speech_string'].astype(str)

In [71]:
corpus_tokenized = [nltk.word_tokenize(sentence) for sentence in meta_df["Cleaned_Speech_string"]]

In [72]:
len(corpus_tokenized)

2583

In [73]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
tagged_docs = [TaggedDocument(words=doc, tags=[i]) for i, doc in enumerate(corpus_tokenized)]

In [74]:
len(tagged_docs)

2583

In [75]:
#TRAINING THE MODEL OVER OUR DATA
model = Doc2Vec(tagged_docs, vector_size=300, window=5, min_count=5, workers=4, epochs=20)

In [76]:
#dumping the model in a pickle so that we do not need to re-run model againas it would yield new set of vectors 
MS_NM_model = open("MS_NM_model.pickle", "wb")
pickle.dump(model, MS_NM_model)
MS_NM_model.close()

In [77]:
import pandas as pd

# Create an empty DataFrame with the desired column names
df_data = pd.DataFrame(columns=['index', 'vector'])

# Iterate over the corpus_tokenized list and generate a vector for each element
for i, speech in enumerate(corpus_tokenized):
    vector = model.infer_vector(speech)
    # Store the index number and vector in a dictionary
    data = {'index': i, 'vector': vector}
    # Add the dictionary as a new row in the DataFrame
    df_data.loc[len(df_data)] = data


In [78]:
df_data

,index,vector
0,0,"[-0.5533242, -0.13845354, -0.045387536, -0.132..."
1,1,"[0.5212499, 0.91167325, -0.65980554, -0.077314..."
2,2,"[0.43338996, 0.52644044, 0.8392834, -0.4008453..."
3,3,"[-0.17666654, 0.43614826, 1.065056, 0.23426825..."
4,4,"[0.46271116, -1.1493033, -0.34250855, 0.453313..."
...,...,...
2578,2578,"[0.60118395, 1.6815414, 2.355218, -0.15670814,..."
2579,2579,"[0.55437744, 0.2630184, -0.21542671, -0.178138..."
2580,2580,"[2.2137816, 1.298251, -0.8109872, 0.4916027, -..."
2581,2581,"[0.6373456, -2.5523167, 2.235605, -1.2152649, ..."


In [79]:
# Extract the vectors as a list from the 'vector' column
vectors = df_data['vector'].tolist()
num_dimensions = len(vectors[0])
num_dimensions

300

In [80]:
# Create column names with unique suffixes
column_names = ['dim_{}'.format(i) for i in range(1, num_dimensions + 1)]

# Create a list of dictionaries representing each row in the new DataFrame
new_data = [{column_names[i]: vector[i] for i in range(num_dimensions)} for vector in vectors]

# Create the new DataFrame
df_new = pd.DataFrame(new_data)
df_new

,dim_1,dim_2,dim_3,dim_4,dim_5,dim_6,dim_7,dim_8,dim_9,dim_10,...,dim_291,dim_292,dim_293,dim_294,dim_295,dim_296,dim_297,dim_298,dim_299,dim_300
0,-0.553324,-0.138454,-0.045388,-0.132011,0.720577,0.250219,-0.587359,0.274449,-0.786303,1.417485,...,-0.153468,-0.639443,-0.956576,1.625992,1.431355,-2.663177,-0.994959,2.307497,1.531194,-1.226561
1,0.521250,0.911673,-0.659806,-0.077315,-1.282651,0.076453,-0.474844,-0.980811,0.674011,1.790803,...,1.181993,-0.596261,2.101596,-0.710893,0.213634,-0.301207,0.449739,-1.218724,0.552643,0.854832
2,0.433390,0.526440,0.839283,-0.400845,0.285049,-0.844874,0.059623,-0.041864,0.486536,0.442388,...,-0.748892,-0.912826,-0.331387,-0.286463,-0.421099,-0.519123,-1.553812,0.512496,0.133026,0.070158
3,-0.176667,0.436148,1.065056,0.234268,1.163537,-0.066943,-0.522028,0.972833,-0.673641,0.855437,...,0.370788,-0.024658,2.319303,-0.812172,-1.711151,-1.796012,0.539112,0.623291,-0.132690,0.276335
4,0.462711,-1.149303,-0.342509,0.453314,-1.561153,-1.173529,-0.694596,0.164316,-0.254244,1.159929,...,0.953166,-0.031245,1.031400,0.502952,0.129597,-0.343925,-0.664846,-0.942154,0.016919,0.285422
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2578,0.601184,1.681541,2.355218,-0.156708,-0.255610,-0.382882,-0.882682,-0.355536,-0.420445,0.241321,...,-1.066611,-1.984733,-0.654360,-2.142338,-0.763030,0.407976,1.039158,-2.210763,2.032165,0.121026
2579,0.554377,0.263018,-0.215427,-0.178139,-0.537649,-0.473560,-2.722322,1.820391,-0.301543,-0.904536,...,-1.251106,-0.091132,0.590598,-0.737561,-0.974242,-0.283213,-0.060491,-1.423453,0.566703,0.541766
2580,2.213782,1.298251,-0.810987,0.491603,-0.561738,-2.083829,-0.124236,2.125675,-0.692431,0.366288,...,-0.845313,-0.755456,-0.381664,2.060044,1.180745,1.924711,-1.069884,-0.544295,0.449652,-2.244463
2581,0.637346,-2.552317,2.235605,-1.215265,0.130622,0.037431,0.698297,-1.907516,-0.340804,1.261990,...,-1.565688,0.852527,0.415011,0.521732,0.789340,0.467500,0.494460,0.124463,-0.291300,-2.642436


In [81]:
MS_NM_vector = open("MS_NM_vector.pickle", "wb")
pickle.dump(df_new,MS_NM_vector)
MS_NM_vector.close()

In [82]:
df_new.insert(0, "index", meta_df["index"])
df_new.insert(1, "Date", meta_df["Date"])

In [83]:
df_new

,index,Date,dim_1,dim_2,dim_3,dim_4,dim_5,dim_6,dim_7,dim_8,...,dim_291,dim_292,dim_293,dim_294,dim_295,dim_296,dim_297,dim_298,dim_299,dim_300
0,MSS1,2004-06-24,-0.553324,-0.138454,-0.045388,-0.132011,0.720577,0.250219,-0.587359,0.274449,...,-0.153468,-0.639443,-0.956576,1.625992,1.431355,-2.663177,-0.994959,2.307497,1.531194,-1.226561
1,MSS2,2004-06-29,0.521250,0.911673,-0.659806,-0.077315,-1.282651,0.076453,-0.474844,-0.980811,...,1.181993,-0.596261,2.101596,-0.710893,0.213634,-0.301207,0.449739,-1.218724,0.552643,0.854832
2,MSS3,2004-07-03,0.433390,0.526440,0.839283,-0.400845,0.285049,-0.844874,0.059623,-0.041864,...,-0.748892,-0.912826,-0.331387,-0.286463,-0.421099,-0.519123,-1.553812,0.512496,0.133026,0.070158
3,MSS4,2004-07-26,-0.176667,0.436148,1.065056,0.234268,1.163537,-0.066943,-0.522028,0.972833,...,0.370788,-0.024658,2.319303,-0.812172,-1.711151,-1.796012,0.539112,0.623291,-0.132690,0.276335
4,MSS5,2004-07-31,0.462711,-1.149303,-0.342509,0.453314,-1.561153,-1.173529,-0.694596,0.164316,...,0.953166,-0.031245,1.031400,0.502952,0.129597,-0.343925,-0.664846,-0.942154,0.016919,0.285422
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2578,NMS1178,2023-04-09,0.601184,1.681541,2.355218,-0.156708,-0.255610,-0.382882,-0.882682,-0.355536,...,-1.066611,-1.984733,-0.654360,-2.142338,-0.763030,0.407976,1.039158,-2.210763,2.032165,0.121026
2579,NMS1179,2023-04-12,0.554377,0.263018,-0.215427,-0.178139,-0.537649,-0.473560,-2.722322,1.820391,...,-1.251106,-0.091132,0.590598,-0.737561,-0.974242,-0.283213,-0.060491,-1.423453,0.566703,0.541766
2580,NMS1180,2023-04-12,2.213782,1.298251,-0.810987,0.491603,-0.561738,-2.083829,-0.124236,2.125675,...,-0.845313,-0.755456,-0.381664,2.060044,1.180745,1.924711,-1.069884,-0.544295,0.449652,-2.244463
2581,NMS1181,2023-04-14,0.637346,-2.552317,2.235605,-1.215265,0.130622,0.037431,0.698297,-1.907516,...,-1.565688,0.852527,0.415011,0.521732,0.789340,0.467500,0.494460,0.124463,-0.291300,-2.642436


In [84]:
MS_NM_index_date_vector_df = open("MS_NM_index_date_vector_df .pickle", "wb")
pickle.dump(df_new, MS_NM_index_date_vector_df )
MS_NM_index_date_vector_df .close()